In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Załadowanie danych z plików CSV
X_data = pd.read_csv('War4_X.csv', sep=';', header=None)
Xprime_data = pd.read_csv('War4_Xprime.csv', sep=';', header=None)

# Konwersja przecinków na kropki i konwersja do float
X_data = X_data.replace(',', '.', regex=True)
Xprime_data = Xprime_data.replace(',', '.', regex=True)

# Konwersja do numpy arrays numerycznych i transpozycja
X = X_data.astype(float).values.T
Xprime = Xprime_data.astype(float).values.T

print("Wymiary macierzy X:", X.shape)
print("Wymiary macierzy X':", Xprime.shape)
print("Typ danych X:", X.dtype)
print("Zakres wartości X: min =", np.min(X), ", max =", np.max(X))

Wymiary macierzy X: (36, 23)
Wymiary macierzy X': (36, 23)


In [5]:
def DMD(X, Xprime, r):
    """
    Dynamic Mode Decomposition
    X: macierz stanów w czasie t
    Xprime: macierz stanów w czasie t+1
    r: rząd aproksymacji
    """
    # Krok 1: SVD macierzy X
    U, Sigma, VT = np.linalg.svd(X, full_matrices=0)
    Ur = U[:, :r]
    Sigmar = np.diag(Sigma[:r])
    VTr = VT[:r, :]
    
    # Krok 2: Obliczenie przybliżonej macierzy A w przestrzeni zredukowanej
    Atilde = np.linalg.solve(Sigmar.T, (Ur.T @ Xprime @ VTr.T).T).T
    
    # Krok 3: Wartości własne i wektory własne Atilde
    Lambda, W = np.linalg.eig(Atilde)
    Lambda = np.diag(Lambda)
    
    # Krok 4: Obliczenie modów DMD
    Phi = Xprime @ np.linalg.solve(Sigmar.T, VTr).T @ W
    
    # Warunki początkowe
    alpha1 = Sigmar @ VTr[:, 0]
    b = np.linalg.solve(W @ Lambda, alpha1)
    
    return Phi, Lambda, b, Atilde

In [6]:
# Zastosowanie DMD do danych
r = 10  # rząd aproksymacji (można eksperymentować)

# Wykonanie DMD
Phi, Lambda, b, Atilde = DMD(X, Xprime, r)

print("Przybliżona macierz A (Atilde) w przestrzeni zredukowanej:")
print("Wymiary Atilde:", Atilde.shape)
print("\nMacierz Atilde:")
print(Atilde)

print("\nWartości własne (Lambda):")
eigenvalues = np.diag(Lambda)
print("Liczba wartości własnych:", len(eigenvalues))
for i, val in enumerate(eigenvalues):
    print(f"λ_{i+1} = {val:.6f}")

UFuncTypeError: Cannot cast ufunc 'svd_s' input from dtype('O') to dtype('float64') with casting rule 'same_kind'

In [ ]:
# Wizualizacja wyników
plt.figure(figsize=(12, 8))

# Wykres 1: Wartości własne na płaszczyźnie zespolonej
plt.subplot(2, 2, 1)
plt.scatter(np.real(eigenvalues), np.imag(eigenvalues), c='red', s=50)
plt.xlabel('Część rzeczywista')
plt.ylabel('Część urojona')
plt.title('Wartości własne na płaszczyźnie zespolonej')
plt.grid(True)

# Wykres 2: Moduły wartości własnych
plt.subplot(2, 2, 2)
magnitudes = np.abs(eigenvalues)
plt.bar(range(len(magnitudes)), magnitudes)
plt.xlabel('Indeks modu')
plt.ylabel('Moduł wartości własnej')
plt.title('Moduły wartości własnych')
plt.grid(True)

# Wykres 3: Macierz Atilde (mapa ciepła)
plt.subplot(2, 2, 3)
plt.imshow(np.real(Atilde), cmap='coolwarm', aspect='auto')
plt.colorbar()
plt.title('Część rzeczywista macierzy Atilde')
plt.xlabel('Kolumna')
plt.ylabel('Wiersz')

# Wykres 4: Wartości osobliwe z SVD
plt.subplot(2, 2, 4)
U, S, VT = np.linalg.svd(X, full_matrices=0)
plt.semilogy(S[:20], 'o-')  # pierwsze 20 wartości
plt.xlabel('Indeks')
plt.ylabel('Wartość osobliwa')
plt.title('Wartości osobliwe macierzy X')
plt.grid(True)

plt.tight_layout()
plt.show()